# YOLOv12m Training - Google Colab
## Setup:
1. Runtime > Change runtime type > GPU (T4)
2. Upload zip file atau mount Google Drive

In [ ]:
# Install ultralytics
!pip install -q ultralytics

## Option 1: Upload Zip File

In [ ]:
# Upload zip file
from google.colab import files
uploaded = files.upload()
# Pilih file: Dataset Parkiran UPJ.v1i.yolov12.zip

In [ ]:
# Extract
!unzip -q "Dataset Parkiran UPJ.v1i.yolov12.zip" -d dataset
!ls -la dataset

## Option 2: Mount Google Drive (Recommended)

In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Upload zip ke Google Drive dulu, lalu extract
# !unzip -q "/content/drive/MyDrive/Dataset Parkiran UPJ.v1i.yolov12.zip" -d dataset

## Training

In [ ]:
from ultralytics import YOLO
import torch

# Check GPU
print(f"CUDA available: {torch.cuda.is_available()}")
print(f"GPU: {torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'None'}")
print(f"GPU count: {torch.cuda.device_count()}")

In [ ]:
# Clear GPU memory
import gc
torch.cuda.empty_cache()
gc.collect()
print(f"GPU memory allocated: {torch.cuda.memory_allocated(0)/1024**3:.2f} GB")
print(f"GPU memory reserved: {torch.cuda.memory_reserved(0)/1024**3:.2f} GB")

In [ ]:
# Load YOLOv12m
model = YOLO('yolo12m.pt')
print("✅ Loaded YOLOv12m")

In [ ]:
# Train
results = model.train(
    data='dataset/data.yaml',
    epochs=150,
    imgsz=1024,  # Turunin dari 1280 biar ga OOM
    batch=4,  # Batch 4 aman untuk T4
    name='yolo12m_parkiran',
    patience=50,
    device=0,
    workers=2,
    cache=False,
    optimizer='AdamW',
    lr0=0.001,
    lrf=0.01,
    momentum=0.937,
    weight_decay=0.0005,
    warmup_epochs=3,
    warmup_momentum=0.8,
    box=7.5,
    cls=0.5,
    dfl=1.5,
    mosaic=1.0,
    mixup=0.1,
    copy_paste=0.1,
    degrees=0.0,
    translate=0.1,
    scale=0.5,
    shear=0.0,
    perspective=0.0,
    flipud=0.0,
    fliplr=0.5,
    hsv_h=0.015,
    hsv_s=0.7,
    hsv_v=0.4,
    max_det=300,
    amp=True,  # Colab support AMP
    verbose=True
)

## Evaluation

In [ ]:
# Validation
print("\n" + "="*50)
print("VALIDATION RESULTS")
print("="*50)
metrics = model.val()
print(f"mAP50: {metrics.box.map50:.4f}")
print(f"mAP50-95: {metrics.box.map:.4f}")
print(f"Precision: {metrics.box.mp:.4f}")
print(f"Recall: {metrics.box.mr:.4f}")
print(f"F1-Score: {2 * (metrics.box.mp * metrics.box.mr) / (metrics.box.mp + metrics.box.mr):.4f}")

In [ ]:
# Per-class metrics
print("\n" + "="*50)
print("PER-CLASS METRICS")
print("="*50)
for i, name in enumerate(model.names.values()):
    print(f"{name}:")
    print(f"  mAP50: {metrics.box.maps[i]:.4f}")
    print(f"  Precision: {metrics.box.p[i]:.4f}")
    print(f"  Recall: {metrics.box.r[i]:.4f}")

In [ ]:
# Test set
print("\n" + "="*50)
print("TEST RESULTS")
print("="*50)
test_metrics = model.val(split='test')
print(f"Test mAP50: {test_metrics.box.map50:.4f}")
print(f"Test mAP50-95: {test_metrics.box.map:.4f}")
print(f"Test Precision: {test_metrics.box.mp:.4f}")
print(f"Test Recall: {test_metrics.box.mr:.4f}")

In [ ]:
# Inference speed
print("\n" + "="*50)
print("INFERENCE SPEED")
print("="*50)
import time
import glob

test_images = glob.glob('dataset/test/images/*')[:10]
if test_images:
    start_time = time.time()
    for img in test_images:
        _ = model.predict(img, verbose=False)
    end_time = time.time()
    avg_time = (end_time - start_time) / len(test_images)
    print(f"Average time: {avg_time*1000:.2f} ms per image")
    print(f"FPS: {1/avg_time:.2f}")

In [ ]:
# Export to ONNX
model.export(format='onnx')
print("\n✅ Model exported to ONNX")

In [ ]:
# Download results
!zip -r results.zip runs/detect/yolo12m_parkiran
files.download('results.zip')
print("\n✅ DONE! Download results.zip")